<br>

# Introdução


In [ ]:
import shutil
import urllib.request
from pathlib import Path

import pandas as pd
from paths import input_path_down, input_path_tab, input_path_tab_temp


<br>

## Functions


In [ ]:
def find_estado(my_url):
    """
    _summary_

    :param my_url: _description_
    :type my_url: _type_
    :return: _description_
    :rtype: _type_
    """
    url_path = Path(my_url)
    url_path_part = url_path.relative_to('https:/geo.fbds.org.br')
    parts = url_path_part.parts[0:-1]
    return parts[0]


In [ ]:
def find_municipio(my_url):
    """
    _summary_

    :param my_url: _description_
    :type my_url: _type_
    :return: _description_
    :rtype: _type_
    """
    url_path = Path(my_url)
    url_path_part = url_path.relative_to('https:/geo.fbds.org.br')
    parts = url_path_part.parts[0:-1]
    return parts[1]


In [ ]:
def find_subfolder(my_url):
    """
    _summary_

    :param my_url: _description_
    :type my_url: _type_
    :return: _description_
    :rtype: _type_
    """
    url_path = Path(my_url)
    url_path_part = url_path.relative_to('https:/geo.fbds.org.br')
    parts = url_path_part.parts[0:-1]
    return parts


In [ ]:
def find_n_subfolder(my_url):
    """
    _summary_

    :param my_url: _description_
    :type my_url: _type_
    :return: _description_
    :rtype: _type_
    """
    url_path = Path(my_url)
    url_path_part = url_path.relative_to('https:/geo.fbds.org.br')
    parts = url_path_part.parts[0:-1]
    return len(parts)


In [ ]:
def find_extension(my_url):
    """
    _summary_

    :param my_url: _description_
    :type my_url: _type_
    :return: _description_
    :rtype: _type_
    """
    url_path = Path(my_url)
    return url_path.suffix


In [ ]:
# find_municipio('https://geo.fbds.org.br/SP/ADAMANTINA/APP/SP_3500105_APP.shp.xml')


<br>

## Read Links


In [ ]:
# List all csv files
list_csv = input_path_tab_temp.rglob('*.csv')

# Read all csv files
df = pd.concat((pd.read_csv(f) for f in list_csv), ignore_index=True)

# Drop Duplicates
df = df.drop_duplicates(subset=df.columns, keep=False)

# Save
df.to_csv(input_path_tab / 'links_final.csv', index=False)


<br>

## Read Dataframe


In [ ]:
# Read Dataframe
df = pd.read_csv(input_path_tab / 'links_final.csv')
df['estado'] = df['url'].apply(find_estado)
df['municipio'] = df['url'].apply(find_municipio)
df['municipio'] = df['municipio'].str.replace('%20', '')
df['extension'] = df['url'].apply(find_extension)
df.sort_values(by='municipio', inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()


<br>

## Estado


In [ ]:
list_estado = list(set(df['estado']))
list_estado.sort()
list_estado


<br>

## Municípios


In [ ]:
# Lista de Municípios
list_mun = list(set(df['municipio']))
list_mun.sort()
print(f'São {len(list_mun)} municíos')
list_mun = list_mun[56:]  # Filtrando
list_mun


In [ ]:
# df_temp = df[df['municipio'] == 'LUIZIANIA']
# df_temp


<br>

## Extensions


In [ ]:
# Lista de Extensões
list_extensions = list(set(df['url']))
list_extensions = [Path(x).suffix for x in list_extensions]
list_extensions = list(set(list_extensions))
list_extensions.sort()
list_extensions.remove('.db')
list_extensions.remove('.lock')
list_extensions.remove('.rar')
list_extensions


In [ ]:
# Avalia Extensões
# df_temp = df[df['extension'] == '.lock']
# df_temp.head()


<br>

## Subfolders


In [ ]:
df['subfolder'] = df['url'].apply(find_subfolder)
df['n_subfolder'] = df['url'].apply(find_n_subfolder)
list(set(df['n_subfolder']))

# df_temp = df[df['n_subfolder'] == 2]
# df_temp


<br>

## _Download_


In [ ]:
for i, row in df.iterrows():
    # Parâmetros
    estado = row['estado']
    municipio = row['municipio']
    extension = row['extension']
    url = row['url']
    n_parts = row['n_subfolder']

    # Só faz download de Extensões relevantes
    if extension in list_extensions:

        # Loop
        for mun in list_mun:
            if mun == municipio:
                # URL
                url_path = Path(url)
                url_path_part = url_path.relative_to('https:/geo.fbds.org.br')

                # Pega Partes
                parts = url_path_part.parts[0:-1]

                # É necessário estar na estrutura ('SP' , 'Município', 'Subfolder')
                if n_parts == 3:
                    down_path_full = (
                        input_path_down / estado / municipio / f'{parts[2]}'
                    )
                    print(down_path_full)
                    down_path_full.mkdir(exist_ok=True, parents=True)
                    filename = down_path_full / url_path.name

                    # Download
                    with urllib.request.urlopen(url) as r, open(
                        filename, 'wb'
                    ) as out_file:
                        shutil.copyfileobj(r, out_file)

# Fim
print('Fim')
